In [1]:
import pandas as pd
from scipy.spatial.distance import cosine
data = pd.read_csv('/Users/mwang/Desktop/System/lastfm.csv')

In [4]:
data.head(6).iloc[:,0:8]

,user,a perfect circle,abba,ac/dc,adam green,aerosmith,afi,air
0,1,0,0,0,0,0,0,0
1,33,0,0,0,1,0,0,0
2,42,0,0,0,0,0,0,0
3,51,0,0,0,0,0,0,0
4,62,0,0,0,0,0,0,0
5,75,0,0,0,0,0,0,0


# Item Based: which takes similarities between items’ consumption histories

In [5]:
# --- Start Item Based Recommendations --- #
# Drop any column named "user"
data_germany = data.drop('user', 1)

In [6]:
# Create a placeholder dataframe listing item vs. item
data_ibs = pd.DataFrame(index=data_germany.columns,columns=data_germany.columns)

In [8]:
# Lets fill in those empty spaces with cosine similarities
# Loop through the columns
for i in range(0,len(data_ibs.columns)):
    # Loop through the columns for each column
    for j in range(0,len(data_ibs.columns)):
      # Fill in placeholder with cosine similarities
      data_ibs.iloc[i,j] = 1-cosine(data_germany.iloc[:,i],data_germany.iloc[:,j])

In [9]:
data_ibs.head()

,a perfect circle,abba,ac/dc,adam green,aerosmith,afi,air,alanis morissette,alexisonfire,alicia keys,...,timbaland,tom waits,tool,tori amos,travis,trivium,u2,underoath,volbeat,yann tiersen
a perfect circle,1,0,0.0179172,0.0515539,0.0627765,0,0.0517549,0.0607177,0,0,...,0.0473381,0.0811998,0.394709,0.125553,0.0303588,0.111154,0.0243975,0.06506,0.0521641,0
abba,0,1,0.0522788,0.0250706,0.0610563,0,0.0167789,0.0295269,0,0,...,0,0,0,0.0610563,0.0295269,0,0.0949158,0,0.0253673,0
ac/dc,0.0179172,0.0522788,1,0.113154,0.177153,0.0678942,0.0757299,0.0380762,0,0.0883332,...,0.0445288,0.0678942,0.0582408,0.0393673,0,0.0871313,0.122398,0.0203997,0.130849,0
adam green,0.0515539,0.0250706,0.113154,1,0.0566365,0,0.0933859,0,0,0.0254164,...,0,0.146516,0.0837892,0.0566365,0.0821687,0.0250706,0.0220113,0,0.023531,0.0880451
aerosmith,0.0627765,0.0610563,0.177153,0.0566365,1,0,0.113715,0.100056,0,0.0618984,...,0.0520051,0.0297351,0.0255072,0.0689655,0.0333519,0,0.214423,0,0.0573068,0


In [11]:
# Create a placeholder items for closes neighbours to an item
data_neighbours = pd.DataFrame(index=data_ibs.columns,columns=range(1,11))

In [45]:
# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(data_ibs.columns)):
    data_neighbours.iloc[i,:10] = data_ibs.iloc[0:,i].sort_values(ascending=False)[0:10].index

# User Based: that considers similarities between user consumption histories and item similarities

In [20]:
# --- Start User Based Recommendations --- #
# Helper function to get similarity scores
def getScore(history, similarities):
    return sum(history*similarities)/sum(similarities)

In [27]:
# Create a place holder matrix for similarities, and fill in the user name column
data_sims = pd.DataFrame(index=data.index,columns=data.columns)
data_sims.iloc[:,:1] = data.iloc[:,:1]

In [ ]:
#Loop through all rows, skip the user column, and fill with similarity scores
for i in range(0,len(data_sims.index)):
    for j in range(1,len(data_sims.columns)):
        user = data_sims.index[i]
        product = data_sims.columns[j]
        print(user, product)
 
        if data.iloc[i][j] == 1:
            data_sims.iloc[i][j] = 0
        else:
            product_top_names = data_neighbours.loc[product][1:10]
            #print(product_top_names)
            product_top_sims = data_ibs.loc[product].sort_values(ascending=False)[1:10]
            #print(product_top_sims)
            user_purchases = data_germany.loc[user,product_top_names]
            #print(user_purchases)
            data_sims.iloc[i][j] = getScore(user_purchases,product_top_sims)

In [53]:
data_sims.head()

,user,a perfect circle,abba,ac/dc,adam green,aerosmith,afi,air,alanis morissette,alexisonfire,...,timbaland,tom waits,tool,tori amos,travis,trivium,u2,underoath,volbeat,yann tiersen
0,1,0,0,0.224335,0,0.290306,0,0.129026,0.202358,0,...,0,0,0,0,0.103175,0,0,0,0,0
1,33,0.088644,0,0.105263,0,0.0969435,0,0.208206,0.189946,0.0747342,...,0.0878403,0.0943826,0,0.093419,0.377637,0,0.102568,0,0.0933201,0.0978537
2,42,0,0.0974134,0,0,0,0,0,0,0,...,0,0,0,0.181926,0,0.117423,0,0,0,0
3,51,0.088644,0.0906869,0,0,0,0.101468,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,62,0,0,0.12545,0.101688,0.0962194,0.101468,0,0.111128,0,...,0.316459,0,0,0,0.0833426,0,0,0,0.100133,0


In [ ]:
# Get the top songs
data_recommend = pd.DataFrame(index=data_sims.index, columns=['user','1','2','3','4','5','6'])
data_recommend.iloc[0:,0] = data_sims.iloc[:,0]

In [60]:
# Instead of top song scores, we want to see names
for i in range(0,len(data_sims.index)):
    data_recommend.iloc[i,1:] = data_sims.iloc[i,:].sort_values(ascending=False).iloc[1:7,].index.transpose()

In [67]:
# Print a sample
print(data_recommend.iloc[:5,:3])

  user                      1              2
0    1         flogging molly       coldplay
1   33  red hot chili peppers  kings of leon
2   42                 oomph!    lacuna coil
3   51            the subways      the kooks
4   62           jack johnson        incubus
